#Building a Neural Network

In [ ]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Predicted class: tensor([2], device='cuda:0')


In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

hidden1 = nn.ReLU()(hidden1)
print(hidden1)

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(model)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
tensor([[0.0000, 0.1324, 0.2550, 0.0000, 0.0000, 0.1552, 0.1867, 0.0000, 0.0104,
         0.2579, 0.0000, 0.0000, 0.0000, 0.0000, 0.1243, 0.0454, 0.0000, 0.1554,
         0.1541, 0.0000],
        [0.0000, 0.0000, 0.2227, 0.0000, 0.0000, 0.1272, 0.2537, 0.0000, 0.1354,
         0.3861, 0.3357, 0.0000, 0.1476, 0.0276, 0.3169, 0.0299, 0.0000, 0.0978,
         0.0000, 0.1253],
        [0.0000, 0.0000, 0.8880, 0.0000, 0.0000, 0.2048, 0.0584, 0.0000, 0.3172,
         0.2065, 0.1278, 0.0000, 0.0000, 0.0000, 0.4764, 0.0780, 0.0418, 0.0518,
         0.0674, 0.0000]], grad_fn=<ReluBackward0>)
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Layer: linear_relu_stack.0

 #Automatic differentation

In [ ]:
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

print(f"Gradient function of the z: {z.grad_fn}")
print(f"Gradient function of the loss: {loss.grad_fn}")

loss.backward()
print(w.grad)
print(b.grad)

z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
  z = torch.matmul(x, w)+b
print(z.requires_grad)

#alternative mode

z = torch.matmul(x,w)+b
z_det = z.detach()
print(z_det.requires_grad)



Gradient function of the z: <AddBackward0 object at 0x77fb3e0707c0>
Gradient function of the loss: <BinaryCrossEntropyWithLogitsBackward0 object at 0x77fc444b4430>
tensor([[0.3109, 0.1071, 0.0019],
        [0.3109, 0.1071, 0.0019],
        [0.3109, 0.1071, 0.0019],
        [0.3109, 0.1071, 0.0019],
        [0.3109, 0.1071, 0.0019]])
tensor([0.3109, 0.1071, 0.0019])
True
False
False


#Optimizing Model Parameters

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()

#optimizer

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#Full implementation

def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)

def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

